In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
from sapy import Clock, ProgramCounter, MemoryAddressRegister, \
    RandomAccessMemory, SwitchBoard, RegisterA, RegisterB, RegisterOutput, \
    ArithmeticUnit, RegisterInstruction, Computer


# Hex and Binary Literals in Python

In [4]:
0xF

15

In [5]:
0xFF

255

In [6]:
print(15_000) # underscore as separators are allowed
print(0b1111_1111)

15000
255


You can also format integers to HEX using X

In [7]:
f"{132:X}"

'84'

# Clocking Signals

In [8]:
clk = Clock()
class DummyComponent():
    def data(self, **kwargs):
        print("data accessed")
        return 42
    def clock(self, **kwargs):
        print("clock stepped")

dc = DummyComponent()
clk.add_component(dc)

In [9]:
print("step 1: ---------")
clk.step()
print("step 2: ---------")
clk.step()

step 1: ---------
data accessed
clock stepped
step 2: ---------
data accessed
clock stepped


# Program Counter

To enable a component to take an action on a clock, you pass in the correct control bit as a keyword argument. To enable the PC to output to the data but, use the "ep" control bit.

In [10]:
pc = ProgramCounter()

In [11]:
pc.clock()
pc.data()

In [12]:
pc.clock()
pc.data(ep=True)

0

In [13]:
pc.clock()
pc.data(ep=True)

0

To increment the program counter, the control bit is "cp".

In [14]:
pc.clock(cp=True)
pc.data(ep=True)

1

In [15]:
pc.clock(cp=True)
pc.data(ep=True)

2

In [16]:
pc.clock(cp=True)
pc.data(ep=True)

3

For jump instructions you set the program counter directly. This is done with the "lp" control bit.

In [17]:
pc.clock(data=0xD, lp=True)
print(f"{pc.data(ep=True):X}")
print(f"{pc.data(ep=True)}")

D
13


# Accumulator (Register A)

In [ ]:
reg_a = RegisterA()

print(f"The register value on reset is: {reg_a.data(ea=True):X}")
reg_a.clock(data=0xAB, la=True)
reg_a.data(ea=True)
print(f"The register has latched a value: {reg_a.data(ea=True):X}")


# Adder

Some components have extra attributes that allow it to communicate directly to other components outside of the bus. 

The adder is not clocked, it doesn't even have state. The data output from the adder is alway the instantaneous sum (or difference) of Reg A + Reg B

In [ ]:
reg_a = RegisterA()
reg_a.clock(data=4, la=True)

reg_b = RegisterB()
reg_b.clock(data=3, lb=True)

adder = ArithmeticUnit(reg_a, reg_b)

assert adder.data() is None
adder.data(eu=True)


In [ ]:
reg_b.clock(data=1, lb=True)
adder.data(eu=True)

Subtraction

In [ ]:
adder.data(eu=True, su=True)

# Memory

Ram is another example of a system that has communication outside the bus, it always access the RAM value stored at the address latched in the Memory Address Register.

In [ ]:
mar = MemoryAddressRegister()
ram = RandomAccessMemory(mar)

# store address for ram in register
mar.clock(data=0xF, lm=True)
print(f"Memory Address: {mar.address():X}")

# clock data into ram at the address set above
ram.clock(data=0xAB, lr=True)
assert ram.data() == None
print(f"Memory at address: {mar.address():X} is {ram.data(er=True):X}") # should be 0xAB
print(f"Memory at address: {mar.address()} is {ram.data(er=True)}") # just to demystfy hex..

# Controller: Sequencer + Decoder + Clock

In [ ]:
reg_i = RegisterInstruction()
instruction = 0xCD # both opcode and argument, 8bits
reg_i.clock(data=instruction, li=True)


In [ ]:
print(f"The full store instruction is {reg_i.value:X}")
print(f"The full store instruction is {reg_i.opcode():X}")
print(f"The memory address to be put on the data bus is {reg_i.data(ei=True):X}")

# Example Sequencing: LDA

In [ ]:
from sapy import Computer

cpu = Computer()
ram.clock(data=0x09, lr=True) # 0x0 LDA 9H

for _ in range(6):
    cpu.step(debug=True) 

In [ ]:
cpu.step(debug=True)

# Output (Register O)

In [ ]:
reg_o = RegisterOutput()
reg_o.clock(data=0x90, lo=True)


In [ ]:
# monkey patch output function for fancy display
def display_number(x):
    print(f"***### {x} ###***")
reg_o.output_function = display_number

In [ ]:
reg_o.clock(data=0x90, lo=True)

# Full Computer with ROM

In [ ]:
import time

In [ ]:
cpu = Computer()
rom = [
    0x04, # 0x0 LDA 4H
    0x15, # 0x1 ADD 5H
    0x30, # 0x2 OUT X
    0x41, # 0x3 JMP 1H
    0x00, # 0x4 A1H
    0x03, # 0x5 22H
    ]
cpu.switches.load_program(rom)

In [ ]:
for _ in range(9):
    cpu.step(instructionwise=True)

In [ ]:
for _ in range(3):
    cpu.step(instructionwise=True, debug=True)

In [ ]:
import ipywidgets as widgets
from IPython.display import display
gui_output_display = widgets.FloatText()
display(gui_output_display)

# monkey patch output function
def display_number(x):
    gui_output_display.value = x

cpu.reg_o.output_function = display_number

In [ ]:
for _ in range(3):
    time.sleep(.2)
    cpu.step(instructionwise=True, debug=False)

In [ ]:
for _ in range(27):
    time.sleep(.03)
    cpu.step(instructionwise=True, debug=False)

In [ ]:
while True:
    time.sleep(.03)
    cpu.step(instructionwise=True, debug=False)